## Estimating individual systematic contributions

The effect of any individual systematic uncertainty is somewhat complicated by it's mutual covariance with the POI (i.e., the W branching fractions) and any other systematic uncertainty.  To get a rough idea of the percent-wise contribution to the total uncertainty from each individual systematic, I use the following scheme:

   * the fit is carried out as in the nominal case and $\sigma_{0}$ is estimated
   * the fit is carried out for for each of the $n$ nuisance parameters $\sigma_{theta}$
   * the difference between the nominal case and the $n-1$ case is calculated,
   * this quantity is normalized to $\sum_{\theta} \sigma_{\theta}^{2}$

In [1]:
## imports and configuration
%cd '/home/naodell/work/wbr/analysis'
#%load_ext autoreload

from multiprocessing import Pool
from functools import partial
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from scipy.optimize import minimize
from tqdm import tqdm_notebook, trange

import scripts.plot_tools as pt
import scripts.fit_helpers as fh

np.set_printoptions(precision=3)
rc_params = {
             'figure.figsize': (10, 10),
             'axes.labelsize': 20,
             'axes.facecolor': 'white',
             'axes.titlesize':'x-large',
             'legend.fontsize': 20,
             'xtick.labelsize':18,
             'ytick.labelsize':18,
             'font.size':18,
             'font.sans-serif':['Arial', 'sans-serif'],
             'mathtext.sf':'Arial',
             'lines.markersize':8.,
             'lines.linewidth':2.5,
            }
matplotlib.rcParams.update(rc_params)

%connect_info

/home/naodell/work/wbr/analysis
{
  "shell_port": 33475,
  "iopub_port": 48967,
  "stdin_port": 45769,
  "control_port": 48079,
  "hb_port": 35355,
  "ip": "127.0.0.1",
  "key": "749639d1-99671e47414144eca9b7ff48",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-73825196-b8b5-4c7a-a6b6-01f99c10ead2.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


In [2]:
# configure, get the input data, and do any additional processing that is needed
processes = ['ttbar', 't', 'ww', 'wjets', 'zjets_alt', 'diboson', 'fakes'] 
selections = [
              'ee',  'mumu',  
              'emu', 
              'mutau', 'etau', 
              'mu4j', 'e4j'
             ]
plot_labels = fh.fancy_labels

# initialize fit data
scenario = 'asimov'
infile = open(f'local_data/fit_data_{asimov}.pkl', 'rb')
fit_data = pickle.load(infile)
infile.close()

# get fit parameters
parameters = fit_data._parameters

In [3]:
# prepare Asimov dataset
asimov_data = {cat:fit_data.mixture_model(parameters.val_init.values, cat) for cat in fit_data._model_data.keys()}

In [4]:
# initialize veto list
fit_data.veto_list = [
    # baseline
    #'ee_cat_gt2_eq1_b', 'ee_cat_gt2_gt2_b', 
    #'mumu_cat_gt2_eq1_b', 'mumu_cat_gt2_gt2_b', 
    #'emu_cat_gt2_eq1_a', 'emu_cat_gt2_gt2_a', 
    #'etau_cat_eq2_eq1', 'etau_cat_gt3_eq1', 'etau_cat_eq2_gt2', 'etau_cat_gt3_gt2', 
    #'mutau_cat_eq2_eq1', 'mutau_cat_gt3_eq1', 'mutau_cat_eq2_gt2', 'mutau_cat_gt3_gt2', 
    #'e4j_cat_gt4_eq1', 'e4j_cat_gt4_gt2',
    #'mu4j_cat_gt4_eq1', 'mu4j_cat_gt4_gt2', 
    'e4j_cat_eq3_gt2', 'mu4j_cat_eq3_gt2',
    
    # e/mu DY CR
    'ee_cat_gt2_eq0',  'mumu_cat_gt2_eq0', 
    
    # e+mu additional ttbar
    #'emu_cat_gt2_eq0', 'emu_cat_eq1_eq0_a', 'emu_cat_eq1_eq1_a', 
    
    # e+mu WW
    #'emu_cat_eq0_eq0_a', 
    
    # e/mu+tau additional CR
    #'mutau_cat_eq0_eq0', 'mutau_cat_eq1_eq0', 
    #'mutau_cat_gt2_eq0', 'mutau_cat_eq1_eq1', 
    #'etau_cat_eq0_eq0', 'etau_cat_eq1_eq0', 
    #'etau_cat_gt2_eq0', 'etau_cat_eq1_eq1', 
]

In [5]:
# fit configuration #

# minimizer options
#steps = 4*[1e-4, ] 
#steps += list(0.05*fit_data._perr_init[4: fit_data._nnorm + 7])
#steps += list(0.05*fit_data._perr_init[fit_data._npoi + fit_data._nnorm:])
#steps = np.array(steps)
min_options = dict(
                   #finite_diff_rel_step=steps,
                   #verbose=3,
                   #eps=1e-9, 
                   gtol = 1e-3,
                   disp = True
                  )

# configure the objective
sample = None
fobj = partial(fit_data.objective,
               data = sample,
               do_bb_lite = True,
               lu_test = None
              )

fobj_jac = partial(fit_data.objective_jacobian,
                   data = sample,
                   do_bb_lite = True,
                   lu_test = None
                  )


In [6]:
# n-1 systematics

# initialize parameter data
fit_data._pmask = parameters['active'].values.astype(bool)
fit_data._pval_init = parameters['val_init'].values.copy()

errs = []
impacts = []
iparam = 4
mask = fit_data._pmask
#mask[17:] = False
p_init = fit_data._pval_fit
p_fit = parameters.val_fit.values
for pname, pdata in tqdm_notebook(parameters.iloc[4:].iterrows(), total=parameters.active.sum() - 4):
    if not pdata.active:
        iparam += 1
        continue
        
    print(iparam, pname, pdata.val_init, pdata.err_init, pdata.val_fit, pdata.err_fit, fobj(p_init[mask]))

    # fix parameter in fit and covariance calculation
    mask[iparam] = False
    #min_options['finite_diff_rel_step'] = steps[mask]
    #err, _ = fh.calculate_covariance(fobj, p_init[mask])
    #errs.append(err)
    
    # calculate impacts from up/down variations of n.p. on p.o.i.
    p_init[iparam] = pdata.val_fit + pdata.err_fit
    print(p_init[iparam], fobj(p_init[mask]))
    res = minimize(fobj, p_fit[mask],
                   jac     = fobj_jac,
                   #hess    = 'cs',
                   #method  = 'trust-constr', 
                   method  = 'BFGS', 
                   options = min_options,
                  )
    impact_up = res.x
    print((res.x[:4]-p_fit[:4])*100/p_fit[:4], res.fun)
                    
    p_init[iparam] = pdata.val_fit - pdata.err_fit
    print(p_init[iparam], fobj(p_init[mask]))
    res = minimize(fobj, p_fit[mask],
                   jac     = fobj_jac,
                   #hess    = 'cs',
                   #method  = 'trust-constr', 
                   method  = 'BFGS', 
                   options = min_options,
                  )
    impact_down = res.x
    print((res.x[:4]-p_fit[:4])*100/p_fit[:4], res.fun)
    impacts.append(np.array([impact_up, impact_down]))
    
    p_init[iparam] = pdata.val_fit
    mask[iparam] = True
    
    iparam += 1
    
errs = np.array(errs)
impacts = np.array(impacts)

/home/naodell/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


4 br_tau_e 0.1774 0.0004 0.17742075818486736 0.00039994783148335444 1515.0446385528924
0.17782070601635072 1517.807215193309


KeyboardInterrupt: 

In [ ]:
# convert impacts to dataframes 
impacts_np = np.hstack([100*(impacts[:,0,:4] - p_fit[:4])/p_fit[:4], 100*(impacts[:,1,:4] - p_fit[:4])/p_fit[:4]])

#impacts_up = pd.DataFrame(impacts_np, columns=['beta_e_up', 'beta_mu_up', 'beta_tau_up', 'beta_h_up'], index=list(p_labels_fancy[4:]))
p_labels = parameters.query('active == 1').index.values[4:]
impacts_np = pd.DataFrame(impacts_np, columns=['beta_e_up', 'beta_mu_up', 'beta_tau_up', 'beta_h_up', 'beta_e_down', 'beta_mu_down', 'beta_tau_down', 'beta_h_down'], index=list(p_labels))
impacts_np.to_csv('local_data/impacts_asimov.csv')

pd.set_option('display.float_format', lambda x: '%.4f' % x)
pd.set_option('display.max_columns', parameters.shape[0])
pd.set_option('display.max_rows', parameters.shape[0])

var = 'beta_tau'
sorted_ind = impacts_np.abs().sort_values(by=f'{var}_down', ascending=False).index
impacts_np.reindex(sorted_ind)[['beta_tau_up', 'beta_tau_down']]

In [ ]:
# convert to errors
err_no_bb = parameters['err_no_bb'].values[mask]
errs_np = np.array([np.concatenate([e[:i+4], [0], e[i+4:]]) for i, e in enumerate(errs)])
errs_np = err_no_bb**2 - errs_np**2
#errs_np[errs_np < 0] = 0
#errs_np = np.sqrt(errs_np)
##errs_np = np.vstack([errs_np, err_stat, err_mc_stat, err_syst, stderr])
#
errs_np = pd.DataFrame(errs_np[:,:4], columns=['beta_e', 'beta_mu', 'beta_tau', 'beta_h'], index=list(p_labels))
##errs_np = pd.DataFrame(errs_np[:,:4], columns=['beta_e', 'beta_mu', 'beta_tau', 'beta_h'], index=list(p_labels_fancy[4:]) + ['stat.', 'MC stat.', 'syst. total', 'total'])
##beta_stderr = stderr.iloc[:,:4].multiply(100)/params_init[:4]
100*errs_np

In [ ]:
# print table

#beta_errs.divide(params_init[:4]/100, axis=1).to_latex('local_data/errors.tex')
#beta_errs.divide(params_init[:4]/100, axis=1).to_csv('local_data/errors.csv')
#beta_errs.divide(params_init[:4]/100, axis=1)
beta_errs.to_latex('local_data/summary_errors.tex', escape=False)
beta_errs

In [ ]:
jes_mask = np.array([True if ('jes' in pname and 'btag' not in pname) else False for pname in beta_errs.index])
btag_mask = np.array([True if 'btag' in pname else False for pname in beta_errs.index])
tau_misid_mask = np.array([True if ('misid_tau' in pname and pname not in ['misid_tau_e', 'misid_tau_h']) else False for pname in beta_errs.index])

btag_errs = beta_errs[btag_mask]
jes_errs = beta_errs[jes_mask]
tau_misid_errs = beta_errs[tau_misid_mask]

summary_errs = beta_errs[~btag_mask&~jes_mask&~tau_misid_mask].copy()
summary_errs.index = [fit_data._parameters.loc[p].label if p in fit_data._parameters.index else p for p in summary_errs.index]
summary_errs.loc['b-tag',:] = np.sqrt(np.sum(btag_errs**2))
summary_errs.loc['JES',:]  = np.sqrt(np.sum(jes_errs**2))
summary_errs.loc[r'$\sf jet\rightarrow\tau$',:]  = np.sqrt(np.sum(tau_misid_errs**2))

summary_errs = summary_errs.divide(params_init[:4]/100, axis=1)
summary_errs.to_latex('local_data/summary_errors.tex', escape=False)
summary_errs.to_csv('local_data/summary_errors.csv')
summary_errs